# Day 12: Rain Risk

Your ferry made decent progress toward the island, but the storm came in faster than anyone expected. The ferry needs to take evasive actions!

Unfortunately, the ship's navigation computer seems to be malfunctioning; rather than giving a route directly to safety, it produced extremely circuitous instructions. When the captain uses the PA system to ask if anyone can help, you quickly volunteer.

The navigation instructions (your puzzle input) consists of a sequence of single-character actions paired with integer input values. After staring at them for a few minutes, you work out what they probably mean:

    Action N means to move north by the given value.
    Action S means to move south by the given value.
    Action E means to move east by the given value.
    Action W means to move west by the given value.
    Action L means to turn left the given number of degrees.
    Action R means to turn right the given number of degrees.
    Action F means to move forward by the given value in the direction the ship is currently facing.

The ship starts by facing east. Only the L and R actions change the direction the ship is facing. (That is, if the ship is facing east and the next instruction is N10, the ship would move north 10 units, but would still move east if the following action were F.)

For example:

```text
F10
N3
F7
R90
F11
```

These instructions would be handled as follows:

    F10 would move the ship 10 units east (because the ship starts by facing east) to east 10, north 0.
    N3 would move the ship 3 units north to east 10, north 3.
    F7 would move the ship another 7 units east (because the ship is still facing east) to east 17, north 3.
    R90 would cause the ship to turn right by 90 degrees and face south; it remains at east 17, north 3.
    F11 would move the ship 11 units south to east 17, south 8.

At the end of these instructions, the ship's Manhattan distance (sum of the absolute values of its east/west position and its north/south position) from its starting position is 17 + 8 = 25.

Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?

In [1]:
# Python imports
from pathlib import Path
from typing import List, Tuple

import numpy as np

As usual, we write a short function to parse input data from a file to a usable form: a list of `<DIR><VAL>` strings. This function could as well have been a list of tuples `(DIR, VAL)`; the choice here means we have to parse each string in any function that takes the loaded route.

In [2]:
def load_route(fpath: str) -> List[str]:
    with Path(fpath).open("r") as ifh:
        return [_.strip() for _ in ifh]

To move the ferry we step through the locations, applying the rules above.

To handle left and right turns, we keep track of a `face` variable that holds the direction the ferry is facing. We use two dictionaries, one for left and one for right, to define the way the ferry ends up facing after a single 90 degree turn in each direction. We have to calculate the number of 90 degree turns the route indicates, in order to use this properly (that's the integer division `val//90`).

We track the location of the ferry after every step, and return the complete route.

In [3]:
def ferry_route(path: List[str], start: Tuple[int, int]=(0,0), face: str="E") -> List[Tuple[int, int]]:
    """Return locations of the ferry along the route indicated in path
    
    :param path:  list of steps in the route as <DIR><VAL> strings
    :param start:  starting location of the ferry as an (x, y) tuple
    :param face:  direction the ferry starts off facing, N, S, E or W
    """
    locations = [(start[0], start[1], face)]
    
    # Dictionaries defining left and right turn effects on ferry facing direction
    rturn = {"N": "E", "E": "S", "S": "W", "W": "N"}
    lturn = {"N": "W", "E": "N", "S": "E", "W": "S"}
    
    # Generate steps in route
    for step in path:
        drn, val = step[:1], int(step[1:])
        curloc = locations[-1]
        
        # Move
        if drn == "F":
            drn = face
        if drn == "N":
            locations.append((curloc[0], curloc[1] + val, face))
        elif drn == "S":
            locations.append((curloc[0], curloc[1] - val, face))
        elif drn == "E":
            locations.append((curloc[0] + val, curloc[1], face))
        elif drn == "W":
            locations.append((curloc[0] - val, curloc[1], face))
            
        # Turn
        elif drn == "R":
            angle = val//90
            for _ in range(angle):
                face = rturn[face]
            locations.append((curloc[0], curloc[1], face))
        elif drn == "L":
            angle = val//90
            for _ in range(angle):
                face = lturn[face]
            locations.append((curloc[0], curloc[1], face))

    return locations

We have a short function that gives the Manhattan distance between the ferry's starting position and the end position, in a route.

In [4]:
def route_distance(route: List[Tuple[int, int]]) -> int:
    """Return Manthattan distance between first and last points on route
    
    :param route:  ferry route as list of (x, y) locations
    """
    start, end = route[0], route[-1]
    return abs(start[0] - end[0]) + abs(start[1] - end[1])

We solve the test puzzle.

In [5]:
path = load_route("day12_test.txt")
route = ferry_route(path)
print(route)
route_distance(route)

[(0, 0, 'E'), (10, 0, 'E'), (10, 3, 'E'), (17, 3, 'E'), (17, 3, 'S'), (17, -8, 'S')]


25

Then solve the real puzzle.

In [6]:
path = load_route("day12_data.txt")
route = ferry_route(path)
route_distance(route)

845

## Part Two

Before you can give the destination to the captain, you realize that the actual action meanings were printed on the back of the instructions the whole time.

Almost all of the actions indicate how to move a waypoint which is relative to the ship's position:

    Action N means to move the waypoint north by the given value.
    Action S means to move the waypoint south by the given value.
    Action E means to move the waypoint east by the given value.
    Action W means to move the waypoint west by the given value.
    Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
    Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
    Action F means to move forward to the waypoint a number of times equal to the given value.

The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

For example, using the same instructions as above:

    F10 moves the ship to the waypoint 10 times (a total of 100 units east and 10 units north), leaving the ship at east 100, north 10. The waypoint stays 10 units east and 1 unit north of the ship.
    N3 moves the waypoint 3 units north to 10 units east and 4 units north of the ship. The ship remains at east 100, north 10.
    F7 moves the ship to the waypoint 7 times (a total of 70 units east and 28 units north), leaving the ship at east 170, north 38. The waypoint stays 10 units east and 4 units north of the ship.
    R90 rotates the waypoint around the ship clockwise 90 degrees, moving it to 4 units east and 10 units south of the ship. The ship remains at east 170, north 38.
    F11 moves the ship to the waypoint 11 times (a total of 44 units east and 110 units south), leaving the ship at east 214, south 72. The waypoint stays 4 units east and 10 units south of the ship.

After these operations, the ship's Manhattan distance from its starting position is 214 + 72 = 286.

Figure out where the navigation instructions actually lead. What is the Manhattan distance between that location and the ship's starting position?

We modify the function above to track the location of the waypoint, relative to the ship. We no longer need to track the way the ship is facing.

Rather than use a dictionary to "move" the waypoint around the ship, we can use [matrix operations](https://en.wikipedia.org/wiki/Rotation_matrix) to rotate its location.

Otherwise, the function is the same as `ferry_route()` except it substitutes the waypoint-related movements for absolute movements.

In [7]:
def waypoint_route(path: List[str], start: Tuple[int, int]=(0,0), wploc: Tuple[int, int]=(10, 1)) -> List[Tuple[int, int]]:
    """Return locations of the ferry along the route indicated in path
    
    :param path:  list of steps in the route as <DIR><VAL> strings
    :param start:  starting location of the ferry as an (x, y) tuple
    :param wploc:  starting location of the waypoint relative to the ferry as an (x, y) tuple
    """
    locations = [start]
    
    # Rotation matrices for left and right turns
    rotl = np.array([[0, -1], [1, 0]])
    rotr = np.array([[0, 1], [-1, 0]])
    
    # Generate steps in route
    for step in path:
        drn, val = step[:1], int(step[1:])
        curloc = locations[-1]

        # Move ferry
        if drn == "F":
            locations.append((curloc[0] + wploc[0] * val, curloc[1] + wploc[1] * val))
        if drn == "N":
            wploc = wploc[0], wploc[1] + val
        elif drn == "S":
            wploc = wploc[0], wploc[1] - val
        elif drn == "E":
            wploc = wploc[0] + val, wploc[1]
        elif drn == "W":
            wploc = wploc[0] - val, wploc[1]
            
        # Rotate waypoint around ferry
        elif drn == "R":
            angle = val//90
            for _ in range(angle):
                wploc = tuple(np.matmul(rotr, np.array(wploc)))
        elif drn == "L":
            angle = val//90
            for _ in range(angle):
                wploc = tuple(np.matmul(rotl, np.array(wploc)))

    return locations

In [8]:
path = load_route("day12_test.txt")
route = waypoint_route(path)
route_distance(route)

286

In [9]:
path = load_route("day12_data.txt")
route = waypoint_route(path)
route_distance(route)

27016